In [1]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import re
import pandas as pd

In [2]:
import mechanize
from bs4 import BeautifulSoup
import urllib
from http import cookiejar


In [ ]:
cj = cookiejar.CookieJar()
br = mechanize.Browser()
br.set_handle_robots(False)
br.set_cookiejar(cj)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]

br.open("https://nation.africa/kenya/account/signin#sign-in")

br.select_form(nr=0)
br.form['email'] = 'tommytsuma7@gmail.com'
br.form['password'] = 'Tobirama13@'
br.submit()



In [3]:
df = pd.DataFrame()

In [4]:
active =[]
links = []
paragraphs = []
times = []
bases = []
images = []

In [5]:

def is_active_url(url):
    """
    Check if the URL is active by sending a HEAD request.
    """
    try:
        response = requests.head(url, allow_redirects=True, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False



In [6]:
def find_all_links(url, soup):
    """
    Find all the hyperlinks in the current page.
    """
    links = set()
    for link in soup.find_all('a', href=True):
        full_url = urljoin(url, link['href'])
        if url in full_url:  # Only consider URLs in the same domain
            links.add(full_url)
    return links



In [7]:
def getPageTime(url):
    response = requests.get(url)
    html_content = response.text
    
    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Check for meta tags related to modification date
    last_modified = soup.find('meta', {'http-equiv': 'last-modified'})
    date_meta = soup.find('meta', {'name': 'date'})
    
    if last_modified:
        return last_modified.get('content')
    elif date_meta:
        return date_meta.get('content')
    else:
        return None


In [8]:
def extract_base_url(url):
    # Regular expression pattern for extracting base URL
    pattern = re.compile(r'^(https?:\/\/[^\/]+)')
    
    # Search for the pattern in the provided URL
    match = pattern.search(url)
    
    # Return the base URL if a match is found, otherwise None
    return match.group(1) if match else None

In [9]:
def getPageText(url):    
    response = requests.get(url)
    html_content = response.text
    
    # Step 2: Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Step 3: Remove Unwanted Tags
    # Find and remove <header>, <footer>, and <nav> tags
    for tag_name in ['header', 'footer', 'nav']:
        for element in soup.find_all(tag_name):
            element.decompose()  # Removes the tag and its content
    
    # Step 4: Extract <p> Tags
    # Find all <p> tags
    paragraphs = soup.find_all('p')
    
    return "\n".join([i.get_text(strip=True) for i in paragraphs])

In [10]:
def getImages(url):
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
    
        # Find all <img> tags on the page
        img_tags = soup.find_all('img')
    
        # Extract the URLs of the images
        img_urls = []
        for img in img_tags:
            img_url = img.get('src')
            # Handle relative URLs by converting them to absolute URLs
            img_url = urljoin(url, img_url)
            img_urls.append(img_url)
    
        # Print the collected image URLs
        print(f"Found {len(img_urls)} images:")
        return img_urls
    else:
        print(f"Failed to retrieve webpage. Status code: {response.status_code}")
        return []

In [11]:
visited_urls = set()

In [12]:
start = time.time()

In [13]:
def crawl_website(base_url):
    global visited_urls, active, links, paragraphs, times, bases, images
    try:
        response = requests.get(base_url, timeout=5)
        if response.status_code == 200:
            print(f"Active URL: {base_url}")
            soup = BeautifulSoup(response.content, 'html.parser')
            current_links = find_all_links(base_url, soup)
            visited_urls.add(base_url)
            paragraphs.append(getPageText(base_url))
            links.append(base_url)
            times.append(getPageTime(base_url))
            bases.append(extract_base_url(base_url))
            images.append(getImages(base_url))
        else:
            print(f"Inactive URL: {base_url} - Status Code: {response.status_code}")
    except requests.RequestException as e:
        print(f"Failed to fetch {base_url}: {e}")

    # for i in [ j for j in current_links if j not in visited_urls]:
    #     if(is_active_url(i)):
    #         # crawl_website(i)
                
    if(len(visited_urls)%100 == 0):
            pd.DataFrame({"links":links, "paragraphs":paragraphs, "time": times, "base_urls":bases,  "img_urls":images}).to_csv("./data/scraped.csv")
   

In [14]:
crawl_website("https://nation.africa/kenya/news/equity-bank-heist-main-suspect-generous-but-secretive--4758616#story")

Active URL: https://nation.africa/kenya/news/equity-bank-heist-main-suspect-generous-but-secretive--4758616#story
Found 12 images:


In [15]:
paragraphs

["Hello\nYour subscription is almost coming to an end. Don’t miss out on the great content on Nation.Africa\nReady to continue your informative journey with us?\nHello\nYour premium access has ended, but the best of Nation.Africa is still within reach. Renew now to unlock exclusive stories and in-depth features.\nReclaim your full access. Click below to renew.\nSubscribe fora month to get full access\nEquity Bank Manager David Machiri Kimani.\nByNyaboga Kiage\n\nIt’s been one month since Mr David Machiri Kimani - the prime suspect in Equity Bank’s Sh1.5 billion heist - was abducted by armed gunmen from his rented house in Thogoto, Kiambu.\nMr Machiri, 39, Equity Bank’s manager at the Group Processing Center, Salary Processing Unit, whose credentials detectives say was used to steal Sh1,545,887,140.49, has never been seen since the Sunday, August 11 abduction.\nAt Madukani, Thogoto under the overpass towards Mutarakwa, is a rough road that leads to an apartment painted purple with a bla

In [ ]:
end = time.time()

In [ ]:
print(end - start)

In [ ]:
df = pd.DataFrame({"links":links[0:20], "paragraphs":paragraphs[0:20], "time": times[0:20], "base_urls":bases[0:20],  "img_urls":images[0:20]})

In [ ]:
df.to_parquet("../data/nation.parquet")

In [ ]:
df.to_csv("../data/nation.csv")

In [ ]:
df